In [0]:
# 1. 데이터 생성 (날짜, 도시, 온도, 습도)
weather_data = [
    ("2023-01-01", "Seoul", -5, 40),
    ("2023-01-01", "Busan", 2, 35),
    ("2023-01-02", "Seoul", -2, 45),
    ("2023-01-02", "Busan", 5, 30),
    ("2023-01-03", "Seoul", 0, 50),
    ("2023-01-03", "Busan", 7, 25)
]

columns = ["Date", "City", "Temp", "Humidity"]

In [0]:
# dataframe 생성
df = spark.createDataFrame(weather_data, schema=columns)

# 데이터 요약을 확인해보자!
df.show()
df.printSchema()

In [0]:
from pyspark.sql.functions import col, when

# 서울의 데이터만 필터링하고 날짜와 온도만 선택해보자
seoul_temp = df.filter(df.City == "Seoul").select("Date", "Temp")
# 온도를 화씨(F)로 변환하는 새 컬럼 추가
df_with_f = df.withColumn("Temp_F", (col("Temp") * 9/5) + 32)

df_status = df_with_f.withColumn("Status", when(col("Temp") < 0, "Cold").otherwise("Mild"))

df_status.show()

In [0]:
from pyspark.sql.functions import avg, max

# 도시별로 그룹화
summary_df = df.groupBy("City").agg(
    avg("Temp").alias("Avg_Temp"),
    max("Humidity").alias("Max_Humidity")
)

summary_df.show()

In [0]:
# filter 또는 where 써도 돼
df_s = df_status.where(df_status.Status == "Mild")

df_status_mild = df_s.groupBy("City").agg(
    avg("Humidity").alias("Avg_Humidity")
)
### 여기까지는 사실 실제로 데이터를 하나도 건드리지 않아.

# Action 단계에서야 비로소 spark가 쌓아둔 계획을 최적화해서 한꺼번에 실행
df_status_mild.show()